In [1]:
# !pip install python-dotenv
# !pip install dotenv
# !pip install PyPDF2

In [2]:
from dotenv import load_dotenv

import os

load_dotenv()
ASU_key = os.environ.get("ASU_key")  

In [19]:
import os

# Specify the folder path
folder_path = "C:/programming_projects/ASU/sarah_pub/" 
#### """/Users/sarahherberger/Documents/ASU/sarah_pub/"""

# List all files and directories in the folder
files = os.listdir(folder_path)

# Print the list of files and directories
for file in files:
    print(file)

def remove_if_starts_with(string_list, char):
    new_list = []
    for string in string_list:
        if not string.startswith(char):
            new_list.append(string)      # Add original string
    return new_list

# Example usage:

files = remove_if_starts_with(files, ".")
print(files)

._AGR-1 AGR-2 and AGR-3 4 Dimensional Change Data Analysis.pdf
._Extracting knowledge from data through catalysis informatics.pdf
._HUMAN FAILURE EVENT DEPENDENCE WHAT ARE THE LIMITS.pdf
._Integration of Human Reliability Analysis Models into the Simulation-Based Framework for the Risk-Informed Safety Margin Characterization Toolkit.pdf
._Multi-Unit Dynamic PRA.pdf
._MULTIVARIATE STATISTICAL ANALYSIS OF COLEOPTERA SPECTRAL REFLECTA.pdf
._Proof-of-Concept Demonstrations for Computation-Based Human Reliability Analysis Modeling Operator Performance During Flooding Scenarios.pdf
._simulated human error prop.pdf
._Testing_Subtask_Quantification_Assumptions_for_Dyn.pdf
._Text-Mining-for-Procedure-Level-Primitives-in-Human-Reliability-Analysis.pdf
AGR-1 AGR-2 and AGR-3 4 Dimensional Change Data Analysis.pdf
DETERMINATION OF A GENERIC HUMAN ERROR PROBABILITY DISTRIBUTION PART 2 A DYNAMIC SPAR-H EXAMPLE APPLICATION.pdf
Extracting knowledge from data through catalysis informatics.pdf
FITTING OF

In [20]:
import pandas as pd
import PyPDF2
import re

page_no = 1

# Open the PDF in read-binary mode
with open(folder_path+files[0], "rb") as pdf_file:
    # Create a PDF reader object
    pdf_reader = PyPDF2.PdfReader(pdf_file)

    # Get the number of pages
    total_page = len(pdf_reader.pages)

    # Extract text from the first page
    page = pdf_reader.pages[page_no]
    text = page.extract_text()

title = re.sub(r'.pdf', ' ', files[1])

import re

def clean_string(text):
    text = re.sub(r'[^A-Za-z0-9., ]', ' ', text)
    text = re.sub(r'  ', ' ', text)
    text = re.sub(r'  ', ' ', text)
    text = re.sub(r'  ', ' ', text)
    return text

cleaned_string = clean_string(text)
cleaned_string

' DISCLAIMER This information was prepared as an account of work sponsored by an agency of the U.S. Government. Ne ither the U.S. Government nor any agency thereof, nor any of their employees, makes any warranty, expressed or implied, or assumes any legal liability or responsibility for the accuracy, completeness, or usefulness, of any information, apparatus, product, or process disclosed, or represents that its use would not infringe privately owned rights. References herein to any specific commercial product, process, or service by trade name, tr ade mark, manufacturer, or otherwise, does not necessarily constitute or imply its endorsement, recommendation, or favoring by the U.S. Government or any agency thereof. The views and opinions of authors expressed herein do not necessarily state or reflect those of the U.S. Government or any agency thereof. '

In [ ]:
from typing import List

def parse_string_with_overlap(text: str, chunk_length: int, overlap_words: int) -> List[str]:
    """
    Parses a string into chunks with a specified length and word overlap.

    Args:
        text: The input string.
        chunk_length: The desired length of each chunk (in words).
        overlap_words: The number of words to overlap between chunks.

    Returns:
        A list of string chunks.  Returns an empty list if the input is invalid.
    """

    if not isinstance(text, str) or not isinstance(chunk_length, int) or not isinstance(overlap_words, int):
        raise TypeError("Input types must be: str, int, int")

    if chunk_length <= 0 or overlap_words < 0 or overlap_words >= chunk_length:
        raise ValueError("chunk_length must be > 0, overlap_words must be >= 0 and < chunk_length")


    words = text.split()
    num_words = len(words)

    if num_words == 0:  # Handle empty string
        return []

    chunks = []
    start_index = 0

    while start_index < num_words:
        end_index = min(start_index + chunk_length, num_words)  # Don't exceed string length
        chunk = " ".join(words[start_index:end_index])
        chunks.append(chunk)

        start_index += (chunk_length - overlap_words)  # Move starting point

    return chunks

word_count = len(cleaned_string.split())

if word_count > 1000:
    try:
        chunks = parse_string_with_overlap(cleaned_string, int(word_count/4), 15)
    except ValueError as e:
        chunks = cleaned_string
        print(f"Error: {e}")
if word_count <= 1000:
    chunks = parse_string_with_overlap(cleaned_string, word_count, 0)

for i, chunk in enumerate(chunks):
    print(f"Chunk {i+1}: {chunk}")

Chunk 1: DISCLAIMER This information was prepared as an account of work sponsored by an agency of the U.S. Government. Ne ither the U.S. Government nor any agency thereof, nor any of their employees, makes any warranty, expressed or implied, or assumes any legal liability or responsibility for the accuracy, completeness, or usefulness, of any information, apparatus, product, or process disclosed, or represents that its use would not infringe privately owned rights. References herein to any specific commercial product, process, or service by trade name, tr ade mark, manufacturer, or otherwise, does not necessarily constitute or imply its endorsement, recommendation, or favoring by the U.S. Government or any agency thereof. The views and opinions of authors expressed herein do not necessarily state or reflect those of the U.S. Government or any agency thereof.


In [18]:
import requests
key = ASU_key

api_url = 'https://api-dev-poc.aiml.asu.edu/queryV2'
bearer_token = key
json_payload = {
    "query": "what part of a document is the following text from in a academic paper {cleaned_string}? only respond with the section type, no other text.".format(cleaned_string=cleaned_string),
    "model_provider": "gcp-deepmind",
    "model_name": "geminiflash1_5",
}
headers = {
    "Authorization": f"Bearer {bearer_token}",
    "Content-Type": "application/json"
}
try:
    response = requests.post(api_url, headers=headers, json=json_payload)
    response.raise_for_status()
    result_document_section = response.json().get("response")
    print("result:", result_document_section)
except requests.exceptions.RequestException as e:
    print(f"API request error: {e}")
except Exception as e:
    print(f"Unexpected error: {e}")

result: Introduction


In [51]:
query = """given that the following text from the document {files} on page {page_no} of total page {total_page}: {cleaned_string}\n\n 
what are some good questions to ask about the {section} section? Please respond with question and answers for 5 questions.

the questions need to be well defined. Do not use the phrase "in the paper" you can say {title}. Answers need to be at least 2 sentences long.

Please use the following format for the response:

**Question 1:**
**Answer 1:**

**Question 2:**
**Answer 2:**

**Question 3:**
**Answer 3:**

**Question 4:**
**Answer 4:**

**Question 5:**
**Answer 5:**
""".format(files=files[1],
                                                                                                                                                                                                                           page_no = page_no,
                                                                                                                                                                                                                           total_page = total_page,
                                                                                                                                                                                                                           cleaned_string=cleaned_string,
                                                                                                                                                                                                                           section=result_document_section,
                                                                                                                                                                                                                           title=title)
print(query)

given that the following text from the document Extracting knowledge from data through catalysis informatics.pdf on page 1 of total page 53: Extracting knowledge from data through catalysis informatics Andrew J. Medford, ,yM. Ross Kunz,zSarah M. Ewing,zTammie Borders,zand Rebecca Fushimi ,z, ySchool of Chemical Biomolecular Engineering, Georgia Institute of Technology, Atlanta, GA, USA zBiological and Chemical Processing Department, Energy and Environmental Science and Technology, Idaho National Laboratory, PO Box 1625, Idaho Falls, ID 83415, USA Center for Advanced Energy Studies, 995 University Boulevard, Idaho Falls, ID 83401, USA E mail andrew.medford chbe.gatech.edu rebecca.fushimi inl.gov Abstract Catalysis informatics is a distinct sub eld that lies at the intersection of cheminformat ics and materials informatics, but with unique challenges arising from the dynamic, surface sensitive, and multi scale nature of heteroge neous catalysis. The ideas behind catalysis in formatics ca

In [52]:
import requests

api_url = 'https://api-dev-poc.aiml.asu.edu/queryV2'
bearer_token = key
json_payload = {
    "query": query,
    "model_provider": "gcp-deepmind",
    "model_name": "geminiflash1_5",
}
headers = {
    "Authorization": f"Bearer {bearer_token}",
    "Content-Type": "application/json"
}
try:
    response = requests.post(api_url, headers=headers, json=json_payload)
    response.raise_for_status()
    result = response.json().get("response")
    print("result:", result)
except requests.exceptions.RequestException as e:
    print(f"API request error: {e}")
except Exception as e:
    print(f"Unexpected error: {e}")

result: Here are 5 questions and answers based on the Abstract of "Extracting knowledge from data through catalysis informatics":

**Question 1:** What are the unique challenges that catalysis informatics faces compared to other informatics disciplines? 

**Answer 1:**  Catalysis informatics faces challenges due to the complex and dynamic nature of heterogeneous catalysis. Unlike typical informatics, this discipline deals with systems where surface sensitivity, multi-scale phenomena, and dynamic interactions are crucial. 

**Question 2:**  How does the concept of "data information knowledge hierarchy" contribute to understanding catalysis informatics?

**Answer 2:** The hierarchy helps categorize different advancements in the field. It allows researchers to understand how raw data is transformed through analysis into meaningful information, ultimately leading to actionable knowledge about catalytic materials.

**Question 3:** What are the key elements of the proposed quantitative repre

In [53]:

parts = result.split("**")  # Split the string at comma
print(parts[2])

parts_no = [[2, 4], [6, 8], [10, 12], [14, 16], [18, 20]]
df_out = pd.DataFrame([])

for i in range(0, 5):
    pt1 = parts_no[i][0]
    pt2 = parts_no[i][1]

    Q1 = pd.DataFrame(data={'section':[result_document_section],
                            'title':[title],
                            'file_name':[files[1]],
                            'document_type':['academic paper'],
                            'subject':['science, chemistry, materials science'],
                            'page': [page_no], 
                            'total_pages': [total_page],
                            'context': [cleaned_string],
                            'question':[parts[pt1]],
                            'answer':[parts[pt2]]
                            })
    df_out = pd.concat([Q1, df_out], ignore_index=True)


In [54]:
df_out

,section,title,file_name,document_type,subject,page,total_pages,context,question,answer
0,Abstract,Extracting knowledge from data through catalys...,Extracting knowledge from data through catalys...,academic paper,"science, chemistry, materials science",1,53,Extracting knowledge from data through catalys...,What is the purpose of including a disclaime...,The disclaimer serves several purposes. It pr...
1,Abstract,Extracting knowledge from data through catalys...,Extracting knowledge from data through catalys...,academic paper,"science, chemistry, materials science",1,53,Extracting knowledge from data through catalys...,Does the disclaimer suggest that the views e...,"No, the disclaimer explicitly states that the..."
2,Abstract,Extracting knowledge from data through catalys...,Extracting knowledge from data through catalys...,academic paper,"science, chemistry, materials science",1,53,Extracting knowledge from data through catalys...,What kind of liability does the U.S. Governme...,The disclaimer states that neither the U.S. G...
3,Abstract,Extracting knowledge from data through catalys...,Extracting knowledge from data through catalys...,academic paper,"science, chemistry, materials science",1,53,Extracting knowledge from data through catalys...,"What specific type of document is ""Extracting...",The disclaimer clarifies that the document is...
4,Abstract,Extracting knowledge from data through catalys...,Extracting knowledge from data through catalys...,academic paper,"science, chemistry, materials science",1,53,Extracting knowledge from data through catalys...,"Who is the sponsor of the work described in ""...",The disclaimer states that the work was prepa...


In [58]:
parts = result.split("**")  # Split the string at comma
print(parts[2])

 What are the unique challenges that catalysis informatics faces compared to other informatics disciplines? 


